In [1]:
import pandas as pd
import numpy as np
import os
from scipy.stats import norm

# --- 1. Setup ---
DATA_DIR = "data"
# Load individual asset returns
asset_returns = pd.read_csv(
    os.path.join(DATA_DIR, "asset_log_returns.csv"),
    index_col='Date',
    parse_dates=True
)
# Define current portfolio weights (equal weight) and confidence level
weights = np.array([1/asset_returns.shape[1]] * asset_returns.shape[1])
alpha = 0.01 # For 99% VaR

print("✅ Setup complete. Asset returns loaded.")

# --- 2. Calculate Portfolio Statistics ---
# Use the full historical covariance matrix and mean returns
cov_matrix = asset_returns.cov()
mean_returns = asset_returns.mean()
portfolio_mean_return = mean_returns.dot(weights)
portfolio_volatility = np.sqrt(weights.T @ cov_matrix @ weights)
z_score = norm.ppf(alpha)

# Calculate total Parametric VaR (expressed as a negative return)
total_var = portfolio_mean_return + portfolio_volatility * z_score

print(f"\nPortfolio Mean Return (Daily): {portfolio_mean_return*100:.4f}%")
print(f"Portfolio Volatility (Daily): {portfolio_volatility*100:.4f}%")
print(f"Z-score for {1-alpha:.0%} VaR: {z_score:.4f}")
print("-" * 40)
print(f"Total Portfolio 99% Parametric VaR: {total_var*100:.4f}%")
print("-" * 40)

✅ Setup complete. Asset returns loaded.

Portfolio Mean Return (Daily): 0.0446%
Portfolio Volatility (Daily): 1.0446%
Z-score for 99% VaR: -2.3263
----------------------------------------
Total Portfolio 99% Parametric VaR: -2.3856%
----------------------------------------
